In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

A keras model will have following methods which are interesting to overwrite:
- 

Example: We want to make a custom CNN model.

We are going to use the classic cifar10 dataset and build a CNN that classifies the images to a label

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

Let's take a look on the data. Always a good idea.

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5 ,5, i + 1)
    plt.xticks([])
    plt.yticks([])    
    plt.imshow(train_images[i])
plt.show()

How does a feature "extraction" extraction looks like with an CNN?

Here is a nice animation:
https://www.youtube.com/watch?v=f0t-OCG79-U

In [ ]:
# Here we are printing the shape of the training and input data

# This is going to be the dimension for the input data
shape_train_images = np.shape(train_images)
print(f'length of train images: {shape_train_images[0]}, shape of train images: {shape_train_images[1:]}')

# This is going to be the dimension for the target data
shape_train_labels = np.shape(train_labels)
print(f'length of train label: {shape_train_labels[0]}, shape of train label: {shape_train_labels[1]}')

# This shows us the length of our data and the dimension
# We are going to need this info for constructing our CNN

# What we can read from the prints is that we have 50000 input 32x32 pixel images with 4 color values for each pixel - RGB
# and 50000 target data with a label

## The Custom Layer
We are goping to inerhit from tf.keras.layers.Layer and build a 'custom' dense layer.<br>
Basically we are building a dense like it's already inplemented in tensorflow.

The best way to build a custom layer is to overwritte these methods if needed:
- __ init __ , here you input-independent initialization
- build, with the shape info of the input you can do the rest of the init
- call, forward computation

A dense layer is nothing other than a elementwise multiplication between the inputs and the kernel

In [ ]:
class CustomDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs, activation):
    super(CustomDenseLayer, self).__init__()
    # here we specify how many units (weights) we are going to have in the layer
    self._num_outputs = num_outputs

  def build(self, input_shape):
    # Here we obviously add the weights to the kerne.
    # For that, we need the info of the dimension/weights/units (input_shape[-1]) from the previous layer
    # and the amopunts of the units/weights (num_outputs) of this dense layer
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self._num_outputs])

  def call(self, inputs):
    # And here we define the element-wise multiplication of the input with the kernel.
    return tf.matmul(inputs, self.kernel)

## The Custom Model
We are going to inerhit from keras.Model and build a 'custom' sequential model.<br>

The best way to build a custom model is to overwritte these methods:
- __ init __ , here you input-independent initialization
- call, forward computation

We could also overwrite methods like fit, build... see documentation for more (https://www.tensorflow.org/api_docs/python/tf/keras/Model).

In [ ]:
class CustomCNN(keras.Model):
    def __init__(self):
        
        super(CustomCNN, self).__init__()
        # First Layer is a 3 dimensional convolutional layer, with 32 filters,
        # where each filter has a kernel size of 3x3
        self._conf_layer_1 = tf.keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation='relu',
            input_shape=(32, 32, 3)
        )
        # With the pooling layer we are reducing the dimension (downsampling the input along its spatial dimensions)
        # by taking the maximum value over an input window, the pool size
        self._pooling_layer_1 = tf.keras.layers.MaxPooling2D((2, 2))
        self._conf_layer_2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        self._pooling_layer_2 = tf.keras.layers.MaxPooling2D((2, 2))
        self._conv_layer_3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')
        # With Flatten we flatten the input (always good to explain the word with the word itself)
        # We are reducing the dimension from (64, 3, 3) to (64). Why. We want to get a propability in one dimension.
        self._flatten = layers.Flatten()
        self._dense = CustomDenseLayer(64, activation='relu')
        # We reduce from 64 to 10. We get 10 numbers who describe to which ot the then classes the picture belongs.
        # So, the probability of which of the 10 classes the image belongs to
        self._dense_2 = CustomDenseLayer(10, activation='linear')
        
    @tf.function
    def call(self, inputs):
        # In the call method we sequentially propagate the input to the former defined layers. 
        x = self._conf_layer_1(inputs)
        x = self._pooling_layer_1(x)
        x = self._conf_layer_2(x)
        x = self._pooling_layer_2(x)
        x = self._conv_layer_3(x)
        x = self._flatten(x)
        x = self._dense(x)
        x = self._dense_2(x)
        return x

In [ ]:
model = CustomCNN()

# 32, 32, 3 like we saw before is our input shape. The None stands for the 'dimension' of how many input data we have
input_shape = (None, 32, 32, 3)

# Here, we build our model
model.build(input_shape=input_shape)

# With model.summary() we can pring the model summry and check it.  
model.summary()

# Now, we compile our model. We are going to pass an optimizer, 
# how the loss is called
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Then, we can fit our model. Here we pass the training data and the validation data
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
# If we don't want to make a custom model, we can build an sequential easy as that:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(CustomDenseLayer(64, activation='relu'))
model.add(CustomDenseLayer(10, activation='linear'))
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Then, we can fit our model. Here we pass the training data and the validation data
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))
# In historty, we have now the information of our model and what happened duiring the training

In [ ]:
# So, we are going to plot the accuracy which was cimputed for the training data
# and the validation data for each epoch

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)